## Deliverable 2. Create a Customer Travel Destinations Map.

In [11]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config2 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [13]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3hrs),Snow inches (last 3hrs)
0,0,Georgetown,MY,2022-04-01 17:23:58,5.4112,100.3354,80.53,88,20,3.44,few clouds,0.0,0.0
1,1,Laguna De Perlas,NI,2022-04-01 17:23:59,12.3429,-83.6712,86.09,70,75,11.45,broken clouds,0.0,0.0
2,2,Ushuaia,AR,2022-04-01 17:22:42,-54.8000,-68.3000,46.02,87,75,19.57,broken clouds,0.0,0.0
3,3,Port Elizabeth,ZA,2022-04-01 17:23:59,-33.9180,25.5701,66.09,78,0,14.97,clear sky,0.0,0.0
4,4,Pacific Grove,US,2022-04-01 17:24:00,36.6177,-121.9166,56.55,86,40,5.01,scattered clouds,0.0,0.0


In [14]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [15]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                      (city_data_df["Max Temp"] >= min_temp)]

In [16]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                    165
City                       165
Country                    164
Date                       165
Lat                        165
Lng                        165
Max Temp                   165
Humidity                   165
Cloudiness                 165
Wind Speed                 165
Current Description        165
Rain inches (last 3hrs)    165
Snow inches (last 3hrs)    165
dtype: int64

In [17]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                    164
City                       164
Country                    164
Date                       164
Lat                        164
Lng                        164
Max Temp                   164
Humidity                   164
Cloudiness                 164
Wind Speed                 164
Current Description        164
Rain inches (last 3hrs)    164
Snow inches (last 3hrs)    164
dtype: int64

In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Georgetown,MY,80.53,few clouds,5.4112,100.3354,
1,Laguna De Perlas,NI,86.09,broken clouds,12.3429,-83.6712,
5,Atuona,PF,78.21,scattered clouds,-9.8000,-139.0333,
7,San Patricio,US,80.69,broken clouds,28.0170,-97.5169,
8,Coatepec,MX,80.37,scattered clouds,19.4500,-96.9667,
9,Hithadhoo,MV,83.55,clear sky,-0.6000,73.0833,
16,Akyab,MM,78.96,few clouds,20.1500,92.9000,
17,Hambantota,LK,80.22,few clouds,6.1241,81.1185,
18,Mahibadhoo,MV,83.95,clear sky,3.7833,72.9667,
27,Teknaf,BD,76.41,clear sky,20.8624,92.3058,


In [21]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [26]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df = hotel_df.replace("", nan_value)
clean_hotel_df = hotel_df.dropna()
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Georgetown,MY,80.53,few clouds,5.4112,100.3354,Cititel Penang
1,Laguna De Perlas,NI,86.09,broken clouds,12.3429,-83.6712,Hotel Best View
5,Atuona,PF,78.21,scattered clouds,-9.8000,-139.0333,Villa Enata
7,San Patricio,US,80.69,broken clouds,28.0170,-97.5169,"Motel 6 Sinton, TX"
8,Coatepec,MX,80.37,scattered clouds,19.4500,-96.9667,Hotel Posada Coatepec
...,...,...,...,...,...,...,...
525,Tayoltita,MX,79.99,clear sky,24.0833,-105.9333,Hotel Rivera Tayoltita
530,Sambava,MG,76.26,light rain,-14.2667,50.1667,Melrose
535,Jalu,LY,82.71,overcast clouds,29.0331,21.5482,صاحبة الفخامة
536,Mangaratiba,BR,76.06,light rain,-22.9597,-44.0406,Portobello Resort & Safari


In [27]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))